In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')


In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
region = ee.FeatureCollection('users/franciscoalbornoz/Aysen')
Map.addLayer(region, {}, 'Aysen')

In [ ]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

In [8]:
start_date = '2020-12-01' # 
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

In [9]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(roi, 10)
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [12]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [13]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [14]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [15]:
greenest = withNDVI.qualityMosaic('NDVI').clip(region)

In [16]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [17]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [18]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [19]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [20]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [21]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [22]:
landcover_2018.bandNames().getInfo()

['b1']

In [23]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [24]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [25]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [26]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [27]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [28]:
work_dir = os.path.expanduser('../Data/shapes/')
achaparrado_shp = os.path.join(work_dir, 'achaparrado_2018_2210_grande.shp')
agricolas_shp = os.path.join(work_dir, 'agricolas_2018_2610_grande.shp')
agua_shp = os.path.join(work_dir, 'agua_2018_0310_grande.shp')
arenalroca_shp = os.path.join(work_dir, 'arenal_roca_2018_0310_grande.shp')
bosquedegradado_shp = os.path.join(work_dir, 'bosques_degradados_2018_22112018.shp')
cuncuna_shp = os.path.join(work_dir, 'cuncunas_2018_2210_grande.shp')
estepa_shp = os.path.join(work_dir, 'estepa_2018_0310_grande.shp')
humedal_shp = os.path.join(work_dir, 'humedal_agua_2018_0310_grande.shp')
mallin_shp = os.path.join(work_dir, 'mallin_2018_0510_grande.shp')
matorralabierto_shp = os.path.join(work_dir, 'matorrales_abiertos_2018_22102018.shp')
matorraldenso_shp = os.path.join(work_dir, 'matorrales_densos_2018_22102018.shp')
plantacion1_shp = os.path.join(work_dir, 'plantaciones_2018_0310_grande.shp')
plantacion2_shp = os.path.join(work_dir, 'plantaciones_2018_23112018_grande.shp')
plantacion3_shp = os.path.join(work_dir, 'plantaciones_2018_26112018_grande.shp')
pradera_shp = os.path.join(work_dir, 'praderas_2018_0310_grande.shp')
primario_shp = os.path.join(work_dir, 'primario_2018_0310_grande.shp')
secundario_shp = os.path.join(work_dir, 'secundario_2018_0310_grande.shp')
sombra_shp = os.path.join(work_dir, 'sombras_topograficas_2018_0310_grande.shp')
suelo_shp = os.path.join(work_dir, 'suelo_desnudo_agricola_2018_0310_grande.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [29]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [30]:
# poligono = ee.FeatureCollection([
#     ee.Feature(achaparrado),
#     ee.Feature(agricolas),
#     ee.Feature(agua),
#     ee.Feature(arenalroca),
#     ee.Feature(bosquedegradado),
#     ee.Feature(cuncuna),
#     ee.Feature(estepa),
#     ee.Feature(humedal),
#     ee.Feature(mallin),
#     ee.Feature(matorralabierto),
#     ee.Feature(matdenso),
#     ee.Feature(plantacion1),
#     ee.Feature(plantacion2),
#     ee.Feature(plantacion3),
#     ee.Feature(pradera),
#     ee.Feature(primario),
#     ee.Feature(secundario),
#     ee.Feature(sombra),
#     ee.Feature(suelo),    
# ])


In [31]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [32]:
dataset = greenest.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [33]:
dataset.size().getInfo()

20322

In [34]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.08925435692071915, 'B10': 292.0626220703125, 'B11': 292.143798828125, 'B2': 0.06809869408607483, 'B3': 0.051249273121356964, 'B4': 0.03142053633928299, 'B5': 0.3350105583667755, 'B6': 0.11977358907461166, 'B7': 0.044714801013469696, 'B8': 0.04191073775291443, 'B9': 0.005282651633024216, 'BQA': 2720, 'ID': 1, 'NDVI': 0.8285050988197327, 'date': 20210105, 'doy': 5, 'month': 1}}


In [35]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
sample = dataset.randomColumn()

split = 0.7 

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [36]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.08925435692071915, 'B10': 292.0626220703125, 'B11': 292.143798828125, 'B2': 0.06809869408607483, 'B3': 0.051249273121356964, 'B4': 0.03142053633928299, 'B5': 0.3350105583667755, 'B6': 0.11977358907461166, 'B7': 0.044714801013469696, 'B8': 0.04191073775291443, 'B9': 0.005282651633024216, 'BQA': 2720, 'ID': 1, 'NDVI': 0.8285050988197327, 'date': 20210105, 'doy': 5, 'month': 1, 'random': 0.5045240732428179}}


In [37]:
print(validation.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_2', 'properties': {'B1': 0.08937954157590866, 'B10': 291.4913024902344, 'B11': 291.53875732421875, 'B2': 0.06789840757846832, 'B3': 0.052150581032037735, 'B4': 0.031620826572179794, 'B5': 0.3308044373989105, 'B6': 0.122127003967762, 'B7': 0.04619194194674492, 'B8': 0.043087445199489594, 'B9': 0.005207542795687914, 'BQA': 2720, 'ID': 1, 'NDVI': 0.8255043029785156, 'date': 20210105, 'doy': 5, 'month': 1, 'random': 0.8763019512832252}}


In [38]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [39]:
# Clasificando con las mismas bandas de entrenamiento
result = greenest.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classificado')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [40]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [41]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [42]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [43]:
land = result.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [44]:
Map.addLayer(landcoverf, {}, 'Land cover Random Forest Región')
Map

Map(center=[-37.30779156528291, -71.17818286898157], controls=(WidgetControl(options=['position', 'transparent…

In [45]:
train_accuracy = classifier.confusionMatrix()

In [46]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 633, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 752, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 3059, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 10, 1605, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 1, 0, 0, 1, 917, 0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 0, 0, 626, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 82, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 223, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 156, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3134, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 250, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 1, 0, 764, 0, 2, 0, 0],
 [0, 0, 0, 11, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 210, 0, 0, 0],
 [0, 0,

In [47]:
train_accuracy.accuracy().getInfo()

0.9941126997476871

In [48]:
train_accuracy.kappa().getInfo()

0.993248592079376

In [49]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9952830188679245],
 [0.9973474801061007],
 [1],
 [0.9931930693069307],
 [1],
 [0.990280777537797],
 [0.9920760697305864],
 [0.9122807017543859],
 [1],
 [1],
 [0.9873417721518988],
 [1],
 [0.9881422924901185],
 [0.9870801033591732],
 [0.9375],
 [0.9910044977511244],
 [0.9415584415584416],
 [0.9938875305623472]]

In [50]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9968503937007874,
  0.9933949801849405,
  0.9915721231766612,
  0.9962756052141527,
  0.9902912621359223,
  0.9839055793991416,
  1,
  1,
  1,
  0.9955357142857143,
  0.9936305732484076,
  0.9977714103788602,
  0.9881422924901185,
  0.9858064516129033,
  1,
  0.9954819277108434,
  1,
  0.9963235294117647]]

In [51]:
validacion = validation.classify(classifier)

In [52]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [53]:
testAccuracy.accuracy().getInfo()

0.964816650148662